<a href="https://colab.research.google.com/github/HandanYU/Rumour-detection/blob/main/obtain_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. split source reply ids

In [172]:
import os
import tqdm
import json
import pandas as pd
from textblob import TextBlob
import pandas as pd
from sklearn import preprocessing
from process import clean_test_data, concat_reply, check_weekday_test, concat_reply_info, extract_stat_tweet_feat
def merge_json(data_type, source_or_reply, ids_list):
    # merged_json: "test_source.json"
    merges_file = os.path.join(f'./tweepy_data/objects/', f'{data_type}_{source_or_reply}.json')
    path_results = f'./tweepy_data/objects/{data_type}_objects'
    with open(merges_file, "w", encoding="utf-8") as f0:
        for file in os.listdir(path_results):
            if file.split('.')[0] in ids_list:
                print('write')
                with open(os.path.join(path_results, file), "r", encoding="utf-8") as f1:
                    for line in tqdm.tqdm(f1):
                        line_dict = json.loads(line)
                        js = json.dumps(line_dict, ensure_ascii=False)
                        f0.write(js + '\n')
                    f1.close()
        f0.close()

def sort_by_time(raw_file, json_file):
    with open(raw_file) as file:
        ids = file.readlines()
    df = pd.read_json(path_or_buf=json_file, lines=True)
    df.index = [str(i) for i in df['id']]
    save_name = raw_file[:-4] + '_sorted.txt'
    with open(save_name, 'w') as file:
        date = pd.Series(pd.DatetimeIndex(df['created_at']), index=df.index)
        df.drop(['created_at'], axis=1, inplace=True)
        df['time'] = date
        for id_ in ids:
            ids_ = id_.strip().split(',')
            source_id = ids_[0]
            file.write(source_id)
            if len(ids_) > 1:
                reply_ids = ids_[1:]
                reply_ids[-1] = reply_ids[-1].strip()
                valid_ids = [index for index in reply_ids if index in df.index]
                sorted_replies = df.loc[valid_ids].sort_values(by='time')
                if len(valid_ids) > 0:
                    file.write(',')

                for i, index in enumerate(sorted_replies.index):
                    file.write(index)
                    if i != len(sorted_replies.index) - 1:
                        file.write(',')

            file.write('\n')
def concat_reply(data_type, source_df):
    """concat replies on source tweets
    data_type: 'dev', 'train', 'test'
    """
    df = pd.DataFrame(columns=['tweet_id', 'reply'])
    with open(f'./tweepy_data/original_data/{data_type}.data_sorted.txt', 'r') as f:
    # with open(f'./data/original_data/{data_type}.data_sorted.txt', 'r') as f:
        content = f.readlines()
    df['tweet_id'] = [c.split(',')[0].strip() for c in content]
    df['reply'] = [','.join([i.strip() for i in c.split(',')[1:]]) for c in content]
    source_df = pd.merge(source_df, df, on='tweet_id', how='left')
    return source_df
def concat_label(data_type, source_feature_df):
    """Concat labels on source tweets
    data_type: 'dev', 'train'
    """
    df = pd.DataFrame(columns=['tweet_id', 'label'])
    with open(f'./tweepy_data/original_data/{data_type}_source.txt', 'r') as f:
        ids = f.readlines()
    with open(f'./tweepy_data/original_data/{data_type}.label.txt', 'r') as f:
        labels = f.readlines()
    df['tweet_id'] = [id.strip() for id in ids]
    df['label'] = [label.strip() for label in labels]
    df_labels = pd.merge(source_feature_df, df, on='tweet_id', how='left')
    df_labels['label'] = df_labels['label'].apply(lambda x: 0 if x == 'nonrumour' else 1)
    return df_labels


def processing(data_type):

    # './data/tweet-objects/test_source.json'
    used_cols = ['created_at', 'id', 'id_str', 'text', 'truncated', 'entities', 'source',
       'in_reply_to_status_id', 'in_reply_to_status_id_str',
       'in_reply_to_user_id', 'in_reply_to_user_id_str',
       'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place',
       'contributors', 'is_quote_status', 'retweet_count', 'favorite_count',
       'favorited', 'retweeted', 'lang', 'extended_entities',
       'possibly_sensitive', 'possibly_sensitive_appealable',
       'quoted_status_id', 'quoted_status_id_str', 'quoted_status']
    source_df = pd.read_json(path_or_buf=f'./tweepy_data/objects/{data_type}_source.json', lines=True)
    source_df = source_df[used_cols]
    reply_df= pd.read_json(path_or_buf=f'./tweepy_data/objects/{data_type}_reply.json', lines=True)
    reply_df = reply_df[used_cols]
    source_df = clean_test_data(source_df)
    reply_df = clean_test_data(reply_df)
    
    
    # get 'verified', 'followers_count', 'listed_count'
    for i in [ 'protected', 'followers_count', 'friends_count', 
                'listed_count', 'favourites_count', 'geo_enabled', 'verified', 
                'statuses_count', 'contributors_enabled']:
        source_df[i] = source_df['user'].apply(lambda x: x[i])
        reply_df[i] = reply_df['user'].apply(lambda x: x[i])
    source_df['has_url'] = source_df['entities'].apply(lambda x: 0 if len(x['urls']) == 0 else 1)
    # get reply statistic info
    reply_df['has_url'] = reply_df['entities'].apply(lambda x: 0 if len(x['urls']) == 0 else 1)
    source_df = concat_reply(data_type, source_df)
    # get reply count
    source_df['reply_count'] = source_df['reply'].apply(lambda x: len(x.split(',')))
    source_df.index = [str(i) for i in source_df['tweet_id']]

    # sorted ids
    if data_type == 'test':
        with open('tweep_data/original_data/test_source.txt', 'r') as f:
            c = f.readlines()
        source_df = source_df.loc[[i.strip() for i in c]]
    source_df = check_weekday_test(source_df)
    reply_df = check_weekday_test(reply_df)
    # add sentiment score
    source_df['senti_score'] = source_df['text'].apply(lambda x: 1 if TextBlob(x).sentiment.polarity > 0 else 0)
    reply_df['senti_score'] = reply_df['text'].apply(lambda x: 1 if TextBlob(x).sentiment.polarity > 0 else 0)
    reply_df.index = [str(i) for i in reply_df['tweet_id']]
    # reply_df = reply_df.rename(columns={'retweet_count': 'retweet_count', 'favorite_count': 'like_count',
    #                                     'mentioned_url_num': 'mentioned_url_num', 'id_num': 'id_num', 'isweekday': 'isweekday'})
    # source_df.index = source_df['tweet_id']
    # source_df = source_df.rename(columns={'retweet_count': 'retweet_count', 'favorite_count': 'like_count', 'followers_count': 'followers_count',
    #                                     'mentioned_url_num': 'mentioned_url_num', 'id_num': 'id_num', 'isweekday': 'isweekday', 
    #                                     'verified': 'verified', 'listed_count': 'tweet_count'})
    # concat replies info to source_df
    # reply_count, quote_count
    count_feat = ['in_reply_to_status_id',
       'in_reply_to_user_id','quoted_status_id']
    statis_feature = [ 'contributors',
       'possibly_sensitive', 'possibly_sensitive_appealable', 'retweet_count', 'favorite_count', 'mentioned_url_num', 'id_num',
       'followers_count', 'friends_count', 'listed_count', 'favourites_count',
       'statuses_count', 'has_url', 'senti_score','truncated', 'is_quote_status', 'favorited', 'retweeted', 'protected',
       'geo_enabled', 'verified', 'contributors_enabled', 'isweekday']
    source_df[['reply_text'] + ['reply_' + s for s in statis_feature]] = source_df.apply(lambda x: concat_reply_info(x['reply'], reply_df, statis_feature), axis=1, result_type='expand')          
    if data_type == 'train':
        source_df = concat_label(data_type, source_df)
    # source_df[['reply_reply_count', 'reply_quote_count', 'quote_count']] = 0      
    return source_df



In [92]:
def split_source_reply(txt_file):
  """
  txt_file: 'train.data.txt'
  """
  with open(txt_file) as f:
      ids = f.readlines()
  source_ids = []
  reply_ids = []
  source_txt_file = txt_file.split('.')[0] + '_source.txt'
  reply_txt_file = txt_file.split('.')[0] + '_reply.txt'
  for i in range(len(ids)):
      source_ids.append(ids[i].split(',')[0].strip())
      reply_ids.extend([r.strip() for r in ids[i].split(',')[1:]])
# save source_ids
  with open(source_txt_file,'w') as f:
      for i in source_ids:
          f.write(i)
          f.write('\n')
# save reply_ids
  with open(reply_txt_file,'w') as f:
      for i in reply_ids:
        f.write(i)
        f.write('\n')


In [175]:
def extract_stat_tweet_feat(istrain, df):
    # extract statistic features
    # reply_reply_count， reply_quote_count，quote_count
    statistic_features = ['reply_' + i for i in ['in_reply_to_status_id', 'in_reply_to_status_id_str',
       'in_reply_to_user_id', 'in_reply_to_user_id_str', 'contributors',
       'possibly_sensitive', 'possibly_sensitive_appealable',
       'quoted_status_id', 'quoted_status_id_str', 'retweet_count', 'favorite_count', 'mentioned_url_num', 'id_num',
       'followers_count', 'friends_count', 'listed_count', 'favourites_count',
       'statuses_count', 'has_url', 'senti_score','truncated', 'is_quote_status', 'favorited', 'retweeted', 'protected',
       'geo_enabled', 'verified', 'contributors_enabled', 'isweekday']] + ['in_reply_to_status_id', 'in_reply_to_status_id_str',
       'in_reply_to_user_id', 'in_reply_to_user_id_str', 'contributors',
       'possibly_sensitive', 'possibly_sensitive_appealable',
       'quoted_status_id', 'quoted_status_id_str', 'retweet_count', 'favorite_count', 'mentioned_url_num', 'id_num',
       'followers_count', 'friends_count', 'listed_count', 'favourites_count',
       'statuses_count', 'has_url', 'senti_score','truncated', 'is_quote_status', 'favorited', 'retweeted', 'protected',
       'geo_enabled', 'verified', 'contributors_enabled', 'isweekday', 'reply_count']
    stat_feat_df = df[statistic_features]
    stat_feat_df.index = df['tweet_id']
    if istrain:
        tweet_df = df[['tweet_id', 'text', 'reply_text', 'label']]
    else:
        tweet_df = df[['tweet_id', 'text', 'reply_text']]
    # tweet_df = df.drop(columns=statistic_features)
    tweet_df.index = df['tweet_id']
    # convert into float
    # for col in ['tweet_count', 'followers_count', 'verified']:
    #     stat_feat_df[col] = stat_feat_df[col].apply(lambda x: float(x))
    # fill nan using corresponding mean
    stat_feat_df = stat_feat_df.fillna(stat_feat_df.mean())
    return stat_feat_df, tweet_df


In [93]:
split_source_reply('tweepy_data/original_data/train.data.txt')

In [ ]:
with open('tweepy_data/original_data/train_source.txt', 'r') as f:
    content = f.readlines()
source_ids = [c.strip() for c in content]
with open('data/original_data/test_reply.txt', 'r') as f:
    content = f.readlines()
reply_ids = [c.strip() for c in content]
merge_json('train', 'source', source_ids)
merge_json('train','reply', reply_ids)

In [113]:
raw_files = ['./tweepy_data/original_data/train.data.txt']
json_files = ['./tweepy_data/objects/train_reply.json']
for raw_file, json_file in zip(raw_files, json_files):
    sort_by_time(raw_file, json_file)

In [134]:
source_df = pd.read_json(path_or_buf=f'./tweepy_data/objects/train_source.json', lines=True)
reply_df= pd.read_json(path_or_buf=f'./tweepy_data/objects/train_reply.json', lines=True)
   

In [142]:
used_cols = ['created_at', 'id', 'id_str', 'text', 'truncated', 'entities', 'source',
       'in_reply_to_status_id', 'in_reply_to_status_id_str',
       'in_reply_to_user_id', 'in_reply_to_user_id_str',
       'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place',
       'contributors', 'is_quote_status', 'retweet_count', 'favorite_count',
       'favorited', 'retweeted', 'lang', 'extended_entities',
       'possibly_sensitive', 'possibly_sensitive_appealable',
       'quoted_status_id', 'quoted_status_id_str', 'quoted_status']
source_df = source_df[used_cols]
reply_df = reply_df[used_cols]
source_df = clean_test_data(source_df)
reply_df = clean_test_data(reply_df)


# get 'verified', 'followers_count', 'listed_count'
for i in [ 'protected', 'followers_count', 'friends_count', 
            'listed_count', 'favourites_count', 'geo_enabled', 'verified', 
            'statuses_count', 'contributors_enabled']:
    source_df[i] = source_df['user'].apply(lambda x: x[i])
source_df['has_url'] = source_df['entities'].apply(lambda x: 0 if len(x['urls']) == 0 else 1)
# get reply statistic info
reply_df['has_url'] = reply_df['entities'].apply(lambda x: 0 if len(x['urls']) == 0 else 1)
source_df = concat_reply('train', source_df)
# get reply count
source_df['reply_count'] = source_df['reply'].apply(lambda x: len(x.split(',')))
source_df.index = [str(i) for i in source_df['tweet_id']]

# sorted ids
# if data_type == 'test':
#     with open('tweep_data/original_data/test_source.txt', 'r') as f:
#         c = f.readlines()
#     source_df = source_df.loc[[i.strip() for i in c]]
source_df = check_weekday_test(source_df)
reply_df = check_weekday_test(reply_df)
# add sentiment score
source_df['senti_score'] = source_df['text'].apply(lambda x: 1 if TextBlob(x).sentiment.polarity > 0 else 0)
reply_df['senti_score'] = reply_df['text'].apply(lambda x: 1 if TextBlob(x).sentiment.polarity > 0 else 0)
reply_df.index = [str(i) for i in reply_df['tweet_id']]


In [173]:
train_df = processing('train')

In [176]:
train_stat_feat_df, train_tweet_df = extract_stat_tweet_feat(True, train_df)

In [8]:
# @timer('ms')
def split_source_reply(txt_file):
  """
  txt_file: 'train.data.txt'
  """
  with open(txt_file) as f:
      ids = f.readlines()
  source_ids = []
  reply_ids = []
  index = 0
  
  for i in range(len(ids)):
      source_ids.append(ids[i].split(',')[0].strip())
      reply_ids.extend([r.strip() for r in ids[i].split(',')[1:]])
      if i > 0 and i % 5000 == 0:
        source_txt_file = txt_file.split('.')[0] + f'_source_data_{index}.txt'
        print(source_txt_file)
        reply_txt_file = txt_file.split('.')[0] + f'_reply_data_{index}.txt'
      # save source_ids
        with open(source_txt_file,'w') as f:
          for i in source_ids:
              f.write(i)
              f.write('\n')
      # save reply_ids
        with open(reply_txt_file,'w') as f:
          for i in reply_ids:
            f.write(i)
            f.write('\n')
        index += 1
        source_ids = []
        reply_ids = []
      elif i == len(ids) - 1:
        source_txt_file = txt_file.split('.')[0] + f'_source_data_{index}.txt'
        print(source_txt_file)
        reply_txt_file = txt_file.split('.')[0] + f'_reply_data_{index}.txt'
      # save source_ids
        with open(source_txt_file,'w') as f:
          for i in source_ids:
              f.write(i)
              f.write('\n')
      # save reply_ids
        with open(reply_txt_file,'w') as f:
          for i in reply_ids:
            f.write(i)
            f.write('\n')
# split_source_reply('dev.data.txt')

In [76]:
def split_source_reply(txt_file):
  """
  txt_file: 'train.data.txt'
  """
  with open(txt_file) as f:
      ids = f.readlines()
  source_ids = []
  reply_ids = []
  source_txt_file = txt_file.split('.')[0] + '_source.txt'
  reply_txt_file = txt_file.split('.')[0] + '_reply.txt'
  for i in range(len(ids)):
      source_ids.append(ids[i].split(',')[0].strip())
      reply_ids.extend([r.strip() for r in ids[i].split(',')[1:]])
# save source_ids
  with open(source_txt_file,'w') as f:
      for i in source_ids:
          f.write(i)
          f.write('\n')
# save reply_ids
  with open(reply_txt_file,'w') as f:
      for i in reply_ids:
        f.write(i)
        f.write('\n')

In [78]:
source_df = pd.read_json(path_or_buf=f'./tweepy_data/objects/test_objects/test_source.json', lines=True)
    

In [ ]:
 'protected', 'followers_count', 'friends_count', 'listed_count', 'favourites_count', 'geo_enabled', 'verified', 'statuses_count', 'contributors_enabled', 

In [89]:
source_df['user'].iloc[0]['geo_enabled']

False

In [80]:
'created_at', 'id', 'text', 'truncated', 'entities', 'source',
       'in_reply_to_status_id', 'in_reply_to_status_id_str',
       'in_reply_to_user_id', 'in_reply_to_user_id_str',
       'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place',
       'contributors', 'is_quote_status', 'retweet_count', 'favorite_count',
       'favorited', 'retweeted', 'lang', 'extended_entities',
       'possibly_sensitive', 'possibly_sensitive_appealable',
       'quoted_status_id', 'quoted_status_id_str', 'quoted_status']

Index(['created_at', 'id', 'id_str', 'text', 'truncated', 'entities', 'source',
       'in_reply_to_status_id', 'in_reply_to_status_id_str',
       'in_reply_to_user_id', 'in_reply_to_user_id_str',
       'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place',
       'contributors', 'is_quote_status', 'retweet_count', 'favorite_count',
       'favorited', 'retweeted', 'lang', 'extended_entities',
       'possibly_sensitive', 'possibly_sensitive_appealable',
       'quoted_status_id', 'quoted_status_id_str', 'quoted_status'],
      dtype='object')

In [74]:
def split_source_reply(txt_file):
  """
  txt_file: 'train.data.txt'
  """
  with open(txt_file) as f:
      ids = f.readlines()
  source_ids = []
  reply_ids = []
  source_txt_file = txt_file.split('.')[0] + '_source.txt'
  reply_txt_file = txt_file.split('.')[0] + '_reply.txt'
  for i in range(len(ids)):
      source_ids.append(ids[i].split(',')[0].strip())
      reply_ids.extend([[r.strip(), ids[i].split(',')[0].strip()] for r in ids[i].split(',')[1:]])
    #   reply_ids.extend([r.strip() for r in ids[i].split(',')[1:]])
# save source_ids
  with open(source_txt_file,'w') as f:
      for i in source_ids:
          f.write(i)
          f.write('\n')
# save reply_ids
  with open(reply_txt_file,'w') as f:
      for i in reply_ids:
        f.write(','.join(i))
        f.write('\n')

# 2. Crawl tweet

In [ ]:
# !twarc2 hydrate  dev.txt > dev_reply_data.jsonl
# resource: https://scholarslab.github.io/learn-twarc/06-twarc-command-basics.html

# 3. Get tweet features

In [12]:
from collections import defaultdict
import json
import pandas as pd

@timer('ms')
def filter_feature(jsonl_file_name):
    """
    jsonl_file_name: 'dev_source_data.jsonl'
    """
    json_file_name = '_'.join(jsonl_file_name.split('_')[:-1]) + '.json'
    json_data = pd.read_json(path_or_buf=jsonl_file_name, lines=True)
    data_dict = defaultdict(dict)
    for i in range(json_data.shape[0]):
        for j in range(len(json_data.data.iloc[i])):
            data_dict[json_data.data.iloc[i][j]['id']]['text'] = json_data.data.iloc[i][j]['text']
            data_dict[json_data.data.iloc[i][j]['id']]['reply_count'] = json_data.data.iloc[i][j]['public_metrics']['reply_count']
            data_dict[json_data.data.iloc[i][j]['id']]['like_count'] = json_data.data.iloc[i][j]['public_metrics']['like_count']
            data_dict[json_data.data.iloc[i][j]['id']]['retweet_count'] = json_data.data.iloc[i][j]['public_metrics']['retweet_count']
            data_dict[json_data.data.iloc[i][j]['id']]['quote_count'] = json_data.data.iloc[i][j]['public_metrics']['quote_count']
            data_dict[json_data.data.iloc[i][j]['id']]['possibly_sensitive'] = json_data.data.iloc[i][j]['possibly_sensitive']
            data_dict[json_data.data.iloc[i][j]['id']]['created_at'] = json_data.data.iloc[i][j]['created_at'] #add create time
            data_dict[json_data.data.iloc[i][j]['id']]['user_id'] = json_data.data.iloc[i][j]['author_id'] #add user id
            data_dict[json_data.data.iloc[i][j]['id']]['has_url'] = 1 if 'entities' in json_data.data.iloc[i][j] else 0 #add shared url number
            

    #  convert into json format
    dict_json=json.dumps(data_dict)
    # save json file
    with open(json_file_name, 'w+') as file:
        file.write(dict_json)
# filter_feature('dev_reply_data.jsonl')

### dump data

In [13]:
jsonls = ['./data/full data/dev_source_data.jsonl',
          './data/full data/dev_reply_data.jsonl',
          './data/full data/train_source_data.jsonl',
          './data/full data/train_reply_data.jsonl']

for jsonl in jsonls:
    filter_feature(jsonl)

Call filter_feature in 203.33385467529297 ms
Call filter_feature in 1865.3488159179688 ms
Call filter_feature in 418.25294494628906 ms
Call filter_feature in 5743.713140487671 ms


## 4. Get user info
#### currently only implement on Source tweet

In [14]:
@timer('ms')
def get_user_info(jsonl_file_name):
    """
    jsonl_file_name: 'dev_source_data.jsonl'
    """
    json_file_name = '_'.join(jsonl_file_name.split('_')[:-1]) + '_userinfo.json'
    json_data = pd.read_json(path_or_buf=jsonl_file_name, lines=True)
    # collect the user info
    info_dict = defaultdict(dict)
    for i in range(json_data.shape[0]):
        for j in range(len(json_data.includes.iloc[i]['users'])):
            info_dict[json_data.includes.iloc[i]['users'][j]['id']]['followers_count'] = json_data.includes.iloc[i]['users'][j]['public_metrics']['followers_count']
            info_dict[json_data.includes.iloc[i]['users'][j]['id']]['tweet_count'] = json_data.includes.iloc[i]['users'][j]['public_metrics']['tweet_count']
            info_dict[json_data.includes.iloc[i]['users'][j]['id']]['verified'] = json_data.includes.iloc[i]['users'][j]['verified']
    #  convert into json format
    dict_json=json.dumps(info_dict)
    # save json file
    with open(json_file_name, 'w+') as file:
        file.write(dict_json)
    
jsonls = ['./data/full data/dev_source_data.jsonl',
          './data/full data/train_source_data.jsonl']

for jsonl in jsonls:
    get_user_info(jsonl)

Call get_user_info in 132.65705108642578 ms
Call get_user_info in 304.43310737609863 ms
